In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd # data processing, CSV file I/O

from xgboost import plot_importance
from matplotlib import pyplot
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
# from sklearn.utils import shuff
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Convolution1D, MaxPooling1D, Flatten, Dropout, concatenate
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
# import seaborn as sns
%matplotlib inline

In [ ]:
X = np.loadtxt('G:/newligo20181220/杏仁分类/workdata/X_xingren.txt')
y = np.loadtxt('G:/newligo20181220/杏仁分类/workdata/y_xingren.txt')
y = y - np.ones(np.size(y))


# X =pd.DataFrame(X,columns=['featr%d'%i for i in range(X.shape[1])])
# X.head()
# y =pd.DataFrame(y)
seed=7
test_size=0.33
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=seed)
print(X_train.shape)
print(X_test.shape)
# X_new = SelectKBest(f_classif, k=1000).fit_transform(X_train, y_train)
m,n=X_train.shape
x = np.zeros((m, n, 1))
for i in range(m):
    for j in range(n):
        x[i, j, 0] = X_train[i, j]
X_train = x

m,n=X_test.shape
x = np.zeros((m, n, 1))
for i in range(m):
    for j in range(n):
        x[i, j, 0] = X_test[i, j]
X_test= x
print(X_train.shape)
print(X_test.shape)

In [ ]:
input1_= Input(shape=(1704, 1), name='input1')
input2_ = Input(shape=(1704, 1), name='input2')

result = Sequential()

add1 = Convolution1D(16, 16, activation='relu', batch_input_shape=(71, 1704, 1))(input1_)
add2 = MaxPooling1D(pool_size=2)(add1)
add3 = Dropout(0.5)(add2)
add4 = Convolution1D(32, 8, activation='relu')(add3)
add5 = MaxPooling1D(pool_size=2)(add4)
add6 = Dropout(0.5)(add5)
add7 = Flatten()(add6)
add8 = Dense(512, activation='relu')(add7)
add9 = Dropout(0.5)(add8)
print(add9)


l1=LSTM(512, return_sequences=True)(input2_)
l2 = Dropout(0.5)(l1)
l3=LSTM(512, return_sequences=True)(l2)
l4 = Dropout(0.5)(l3)
l5=LSTM(512)(l4)
l6 = Dropout(0.5)(l5)
print(l6)


merged = concatenate([add9, l6])

# result.add(merged)
output = Dense(7, activation='softmax')(merged)

model = Model(inputs=[input1_, input2_], outputs=output)
model.summary()


model.compile(loss='categorical_crossentropy', optimizer='adam')
# 神经网络建立完毕

y_train = to_categorical(y_train, 7)
y_test = to_categorical(y_test, 7)

model.fit([X_train,X_train], y_train, batch_size=71, epochs=200)

f.mkdir('model')
path = 'model' + '/' + 'model_多分类_multi.h5'
model.save(path)
print('训练完成，训练后的模型保存在：' + path + '文件中')